In [1]:
import requests
import pandas as pd

def get_ntesstudysi(session, course_url):
    """
    请求课程页面并提取 NTESSTUDYSI Cookie 值。

    :param session: requests.Session 对象，用于保持会话。
    :param course_url: 课程页面的 URL。
    :return: NTESSTUDYSI 的值。
    """
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/128.0.0.0 Safari/537.36"
        ),
        "Accept": (
            "text/html,application/xhtml+xml,application/xml;"
            "q=0.9,image/webp,image/apng,*/*;q=0.8"
        ),
        "Accept-Language": "zh-CN,zh;q=0.9",
    }

    response = session.get(course_url, headers=headers)
    response.raise_for_status()

    # 提取 NTESSTUDYSI Cookie
    ntesstudysi = session.cookies.get('NTESSTUDYSI')
    if not ntesstudysi:
        raise ValueError("未找到 NTESSTUDYSI Cookie。请检查请求是否成功或 Cookie 名称是否正确。")

    return ntesstudysi

def make_rpc_request(session, rpc_url_base, ntesstudysi, payload):
    """
    使用 NTESSTUDYSI 作为 csrfKey 发起 RPC 请求。

    :param session: requests.Session 对象，用于保持会话。
    :param rpc_url_base: RPC 请求的基础 URL（不包含 csrfKey）。
    :param ntesstudysi: NTESSTUDYSI 的值，用作 csrfKey。
    :param payload: POST 请求的负载（字符串格式）。
    :return: RPC 请求的 JSON 响应。
    """
    # 构建完整的 RPC 请求 URL，包含 csrfKey 参数
    rpc_url = f"{rpc_url_base}?csrfKey={ntesstudysi}"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/128.0.0.0 Safari/537.36"
        ),
        "Accept": "*/*",
        "Accept-Language": "zh-CN,zh;q=0.9",
        "Content-Type": "application/x-www-form-urlencoded",
        "Priority": "u=1, i",
        "Sec-CH-UA": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Referer": "https://www.icourse163.org/course/NHDX-1463126169",
        "Origin": "https://www.icourse163.org",
        "Connection": "keep-alive",
    }

    data = payload  # 负载为字符串格式，例如 "courseId=1463126169&pageIndex=1&pageSize=20&orderBy=3"

    response = session.post(rpc_url, headers=headers, data=data)
    response.raise_for_status()

    # 尝试解析 JSON 响应
    try:
        return response.json()
    except ValueError:
        raise ValueError("响应不是有效的 JSON 格式。")

def parse_comments(json_response):
    """
    解析 RPC 请求的 JSON 响应并提取评论数据。

    :param json_response: RPC 请求的 JSON 响应。
    :return: 提取到的评论列表。
    """
    comments = []
    if json_response and 'result' in json_response and 'list' in json_response['result']:
        for comment in json_response['result']['list']:
            comment_data = {
                'comment_id': comment.get('id'),
                'user_nickname': comment.get('userNickName'),
                'content': comment.get('content'),
                'agree_count': comment.get('agreeCount'),
                'mark': comment.get('mark'),
                'gmt_modified': comment.get('gmtModified')
            }
            comments.append(comment_data)
    return comments

def save_comments_to_csv(comments, file_name):
    """
    将提取的评论数据保存到 CSV 文件。

    :param comments: 评论数据列表。
    :param file_name: 保存的 CSV 文件名。
    """
    df = pd.DataFrame(comments)
    df.to_csv(file_name, index=False, encoding='utf-8-sig')
    print(f"评论数据已保存到 {file_name}")

def main():
    course_url = "https://www.icourse163.org/course/NHDX-1463126169"
    rpc_url_base = "https://www.icourse163.org/web/j/mocCourseV2RpcBean.getCourseEvaluatePaginationByCourseIdOrTermId.rpc"

    # POST 请求的负载
    payload = "courseId=1463126169&pageIndex=1&pageSize=20&orderBy=3"

    with requests.Session() as session:
        try:
            # 步骤 1：请求课程页面并提取 NTESSTUDYSI Cookie
            ntesstudysi = get_ntesstudysi(session, course_url)
            print(f"提取到的 NTESSTUDYSI: {ntesstudysi}")

            # 步骤 2：使用 NTESSTUDYSI 作为 csrfKey 发起 RPC 请求
            response = make_rpc_request(session, rpc_url_base, ntesstudysi, payload)
            print("RPC 请求响应：")
            print(response)

            # 步骤 3：解析评论数据
            comments = parse_comments(response)

            # 步骤 4：将解析后的评论数据保存到 CSV 文件
            save_comments_to_csv(comments, 'course_comments.csv')

        except Exception as e:
            print(f"发生错误: {e}")

if __name__ == "__main__":
    main()


提取到的 NTESSTUDYSI: f710c8fa48fc4a8ea701328caf3f037a
RPC 请求响应：
{'code': 0, 'result': {'query': {'sortCriterial': ' agree_count desc, gmt_modified desc,mark desc ', 'DEFAULT_PAGE_SIZE': 10, 'DEFAULT_PAGE_INDEX': 1, 'DEFAULT_TOTLE_PAGE_COUNT': 1, 'DEFAULT_TOTLE_COUNT': 0, 'DEFAULT_OFFSET': 0, 'pageSize': 20, 'pageIndex': 1, 'totlePageCount': 73, 'totleCount': 1446, 'offset': 0, 'limit': 20}, 'list': [{'id': 6753726, 'gmtModified': 1705826239453, 'commentorId': 1021440455, 'userNickName': 'jjxl2018', 'faceUrl': 'https://img-ph-mirror.nosdn.127.net/25AeHHpyQV9JxQ3BIpguKA==/6632196964140525554.jpg', 'content': '我是一名医学基础课程老师，负责病理生理学的教学，为了适应新时代的需求，我们学校开设了一个新兴医学专业“医 X”。在新医科以人工智能、大数据、云计算为代表的科技革命新背景下，本专业在病理生理学教学课程中也融入人工智能、大数据等相关内容。李老师的计算机课不仅给学生带来新的技术技能，也给我们带来很多启发，为实现培养“学科交叉、服务临床、注重创新、引领发展”的复合型医学人才做好了充分的准备。', 'mark': 5.0, 'courseId': None, 'termId': 1470746442, 'courseName': None, 'agreeCount': 22, 'status': 1, 'upvote': None, 'productType': None, 'shortName': None, 'courseMode': None}, {'id': 6622

In [3]:
import requests
import pandas as pd

def get_ntesstudysi(session, course_url):
    """
    请求课程页面并提取 NTESSTUDYSI Cookie 值。

    :param session: requests.Session 对象，用于保持会话。
    :param course_url: 课程页面的 URL。
    :return: NTESSTUDYSI 的值。
    """
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/128.0.0.0 Safari/537.36"
        ),
        "Accept": (
            "text/html,application/xhtml+xml,application/xml;"
            "q=0.9,image/webp,image/apng,*/*;q=0.8"
        ),
        "Accept-Language": "zh-CN,zh;q=0.9",
    }

    response = session.get(course_url, headers=headers)
    response.raise_for_status()

    # 提取 NTESSTUDYSI Cookie
    ntesstudysi = session.cookies.get('NTESSTUDYSI')
    if not ntesstudysi:
        raise ValueError("未找到 NTESSTUDYSI Cookie。请检查请求是否成功或 Cookie 名称是否正确。")

    return ntesstudysi

def make_rpc_request(session, rpc_url_base, ntesstudysi, payload):
    """
    使用 NTESSTUDYSI 作为 csrfKey 发起 RPC 请求。

    :param session: requests.Session 对象，用于保持会话。
    :param rpc_url_base: RPC 请求的基础 URL（不包含 csrfKey）。
    :param ntesstudysi: NTESSTUDYSI 的值，用作 csrfKey。
    :param payload: POST 请求的负载（字符串格式）。
    :return: RPC 请求的 JSON 响应。
    """
    # 构建完整的 RPC 请求 URL，包含 csrfKey 参数
    rpc_url = f"{rpc_url_base}?csrfKey={ntesstudysi}"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/128.0.0.0 Safari/537.36"
        ),
        "Accept": "*/*",
        "Accept-Language": "zh-CN,zh;q=0.9",
        "Content-Type": "application/x-www-form-urlencoded",
        "Priority": "u=1, i",
        "Sec-CH-UA": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Referer": "https://www.icourse163.org/course/NHDX-1463126169",
        "Origin": "https://www.icourse163.org",
        "Connection": "keep-alive",
    }

    response = session.post(rpc_url, headers=headers, data=payload)
    response.raise_for_status()

    # 尝试解析 JSON 响应
    try:
        return response.json()
    except ValueError:
        raise ValueError("响应不是有效的 JSON 格式。")

def parse_comments(json_response):
    """
    解析 RPC 请求的 JSON 响应并提取评论数据。

    :param json_response: RPC 请求的 JSON 响应。
    :return: 提取到的评论列表。
    """
    comments = []
    if json_response and 'result' in json_response and 'list' in json_response['result']:
        for comment in json_response['result']['list']:
            comment_data = {
                'comment_id': comment.get('id'),
                'user_nickname': comment.get('userNickName'),
                'content': comment.get('content'),
                'agree_count': comment.get('agreeCount'),
                'mark': comment.get('mark'),
                'gmt_modified': comment.get('gmtModified')
            }
            comments.append(comment_data)
    return comments

def save_comments_to_csv(comments, file_name):
    """
    将提取的评论数据保存到 CSV 文件。

    :param comments: 评论数据列表。
    :param file_name: 保存的 CSV 文件名。
    """
    df = pd.DataFrame(comments)
    df.to_csv(file_name, index=False, encoding='utf-8-sig')
    print(f"评论数据已保存到 {file_name}")

def main():
    course_url = "https://www.icourse163.org/course/NHDX-1463126169"
    rpc_url_base = "https://www.icourse163.org/web/j/mocCourseV2RpcBean.getCourseEvaluatePaginationByCourseIdOrTermId.rpc"
    total_pages = 74  # 总页数
    all_comments = []  # 用于存储所有页的评论

    with requests.Session() as session:
        try:
            # 步骤 1：请求课程页面并提取 NTESSTUDYSI Cookie
            ntesstudysi = get_ntesstudysi(session, course_url)
            print(f"提取到的 NTESSTUDYSI: {ntesstudysi}")

            # 步骤 2：循环请求每一页的评论数据
            for page in range(1, total_pages + 1):
                payload = f"courseId=1463126169&pageIndex={page}&pageSize=20&orderBy=3"
                response = make_rpc_request(session, rpc_url_base, ntesstudysi, payload)
                print(f"正在提取第 {page} 页评论数据...")

                # 解析每一页的评论并将其加入 all_comments 列表
                comments = parse_comments(response)
                all_comments.extend(comments)

            # 步骤 3：将所有页的评论数据保存到 CSV 文件
            save_comments_to_csv(all_comments, 'all_course_comments.csv')

        except Exception as e:
            print(f"发生错误: {e}")

if __name__ == "__main__":
    main()


提取到的 NTESSTUDYSI: 8827cb65af0b4ecda22d834c3d66e715
正在提取第 1 页评论数据...
正在提取第 2 页评论数据...
正在提取第 3 页评论数据...
正在提取第 4 页评论数据...
正在提取第 5 页评论数据...
正在提取第 6 页评论数据...
正在提取第 7 页评论数据...
正在提取第 8 页评论数据...
正在提取第 9 页评论数据...
正在提取第 10 页评论数据...
正在提取第 11 页评论数据...
正在提取第 12 页评论数据...
正在提取第 13 页评论数据...
正在提取第 14 页评论数据...
正在提取第 15 页评论数据...
正在提取第 16 页评论数据...
正在提取第 17 页评论数据...
正在提取第 18 页评论数据...
正在提取第 19 页评论数据...
正在提取第 20 页评论数据...
正在提取第 21 页评论数据...
正在提取第 22 页评论数据...
正在提取第 23 页评论数据...
正在提取第 24 页评论数据...
正在提取第 25 页评论数据...
正在提取第 26 页评论数据...
正在提取第 27 页评论数据...
正在提取第 28 页评论数据...
正在提取第 29 页评论数据...
正在提取第 30 页评论数据...
正在提取第 31 页评论数据...
正在提取第 32 页评论数据...
正在提取第 33 页评论数据...
正在提取第 34 页评论数据...
正在提取第 35 页评论数据...
正在提取第 36 页评论数据...
正在提取第 37 页评论数据...
正在提取第 38 页评论数据...
正在提取第 39 页评论数据...
正在提取第 40 页评论数据...
正在提取第 41 页评论数据...
正在提取第 42 页评论数据...
正在提取第 43 页评论数据...
正在提取第 44 页评论数据...
正在提取第 45 页评论数据...
正在提取第 46 页评论数据...
正在提取第 47 页评论数据...
正在提取第 48 页评论数据...
正在提取第 49 页评论数据...
正在提取第 50 页评论数据...
正在提取第 51 页评论数据...
正在提取第 52 页评论数据...
正在提取第 53 页评论数据...
正在提取